In [1]:
import pandas as pd
i="アニメゲーム"
df=pd.read_csv(f"../Data/{i}.csv")
df
# 열 Q, K1, K2, K3, K4의 내용을 합쳐서 하나의 열로 합치기
df['QK'] = df['Q'] +"? : 1."+ df['K1'] +", 2."+  df['K2'] + ", 3."+ df['K3'] +", 4."+  df['K4']
df=df[['QK','A']]
#열 이름을 text, prediction으로 고치기
df.columns=['text','prediction']
df


,text,prediction
0,1997年に公開された映画『シト新生』は、何というアニメの劇場版作ロ\r\nロロ ?? : ...,機動警察パトレイバー
1,アニメ『ゲゲゲの鬼太郎』で、鬼太郎に助けて欲しい時に手紙を入れるボストの名前は何ポスト ??...,悪霊ポスト
2,その展開がよく「起承転結」というーー >ニス混面ビリロリテリ\r\n四字熟語で例えられる漫...,4コマ漫画
3,1990年に発売された、「ファミリーコンピュータ」の後継機であるゲーム機は「何ファミコン」 ...,ミラクルファミコン
4,"石川チカの漫画 !交番PB』に登場するコンビ、仙波と武田の職業は何 ?? : 1.郵便局員,...",韻察到
...,...,...
2813,"の だゴクEi【ひ28較=レン舞台にした作品はどれ ?? : 1.イヴの有眠り, 2.カリフ...",海街diary
2814,"アニメ 「ルパン三世』に登場するルパンの仲間、五工門の名字は何 ?? : 1.次元, 2.峰...",石川
2815,小林よしのりの漫画『おぼっちゃまくん』で、おぼっちゃまくんは「歩く何」と呼ばれていた ?? ...,身代金
2816,"次のうち、魔法使いの少女を主人公とする漫画はどれ2?? : 1.アルスラーン戦記, 2.将棋...",ふらいんぐうつういっち


In [2]:
import numpy as np
from datasets import load_dataset

train, valid, test = df,df.loc[:10],df.loc[:10]
print(f"Source News : {train.text.iloc[0][:200]}")
print(f"Summarization : {train.prediction.iloc[0][:50]}")
print(f"Training Data Size : {len(train)}")
print(f"Validation Data Size : {len(valid)}")
print(f"Testing Data Size : {len(test)}")

Source News : 1997年に公開された映画『シト新生』は、何というアニメの劇場版作ロ
ロロ ?? : 1.機動警察パトレイバー, 2.新世紀エヴァンゲリオン, 3.交響詩馬エウレカセブン, 4.コードギアス
Summarization : 機動警察パトレイバー
Training Data Size : 2818
Validation Data Size : 11
Testing Data Size : 11


In [3]:
import torch
from transformers import BartTokenizer
from torch.utils.data import TensorDataset, DataLoader
from torch.utils.data import RandomSampler, SequentialSampler
from torch.nn.utils.rnn import pad_sequence


def make_dataset(data, tokenizer, device):
    tokenized = tokenizer(
        text=data.text.tolist(),
        padding="longest",
        truncation=True,
        return_tensors="pt"
    )
    labels = []
    input_ids = tokenized["input_ids"].to(device)
    attention_mask = tokenized["attention_mask"].to(device)
    for target in data.prediction:
        labels.append(tokenizer.encode(target, return_tensors="pt").squeeze())
    labels = pad_sequence(labels, batch_first=True, padding_value=-100).to(device)
    return TensorDataset(input_ids, attention_mask, labels)



def get_datalodader(dataset, sampler, batch_size):
    data_sampler = sampler(dataset)
    dataloader = DataLoader(dataset, sampler=data_sampler, batch_size=batch_size)
    return dataloader


epochs = 50
batch_size = 8
device = "cuda" if torch.cuda.is_available() else "cpu"
tokenizer = BartTokenizer.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base"
)

train_dataset = make_dataset(train, tokenizer, device)
train_dataloader = get_datalodader(train_dataset, RandomSampler, batch_size)

valid_dataset = make_dataset(valid, tokenizer, device)
valid_dataloader = get_datalodader(valid_dataset, SequentialSampler, batch_size)

test_dataset = make_dataset(test, tokenizer, device)
test_dataloader = get_datalodader(test_dataset, SequentialSampler, batch_size)

print(train_dataset[0], device)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(tensor([    0, 40954, 47168, 20024, 48016, 45895, 11582, 49035, 13859, 49173,
        48916, 48179, 48647, 21402, 47873,  2023, 49844, 49285, 49280, 47240,
         7487, 48998, 49839, 48549, 47341, 47856, 15722, 48281, 47780, 48718,
        49017, 49464, 49578, 49223, 27969,  6382, 42393, 21402, 20024, 49559,
        49233, 49323, 50121, 50118, 49323, 49323, 42254,  4832,   112,     4,
        49449, 48675, 15722, 36484, 12410, 18164, 48420,  4333, 49630, 49280,
        49390, 48611, 49441, 48438,     6,   132,     4, 47240,  7487, 46015,
        25448, 36714, 20024,  7471, 49682, 50033, 48078, 47783, 14292, 48947,
        49502, 48078,     6,   155,     4, 46499, 10470, 41907,  4333,  9470,
        36484, 15375, 15375, 41907, 18164, 11582, 49682, 49422, 49390, 49063,
        49754, 49542, 48078,     6,   204,     4, 49195, 48943, 23171, 49904,
        49017, 48791,     2,     1,     1,     1,     1,     1,     1,     1,
            1,     1,     1,     1,     1,     1,     1,     1,

In [4]:
from torch import optim
from transformers import BartForConditionalGeneration


model = BartForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base"
).to(device)
model.load_state_dict(torch.load(f"../Model/Bart_{i}.pt"))
optimizer = optim.AdamW(model.parameters(), lr=5e-5, eps=1e-8)

In [5]:
for main_name, main_module in model.named_children():
    print(main_name)
    for sub_name, sub_module in main_module.named_children():
        print("└", sub_name)
        for ssub_name, ssub_module in sub_module.named_children():
            print("│  └", ssub_name)
            for sssub_name, sssub_module in ssub_module.named_children():
                print("│  │  └", sssub_name)

model
└ shared
└ encoder
│  └ embed_tokens
│  └ embed_positions
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  │  └ 3
│  │  └ 4
│  │  └ 5
│  └ layernorm_embedding
└ decoder
│  └ embed_tokens
│  └ embed_positions
│  └ layers
│  │  └ 0
│  │  └ 1
│  │  └ 2
│  │  └ 3
│  │  └ 4
│  │  └ 5
│  └ layernorm_embedding
lm_head


In [6]:
import numpy as np
import evaluate
from tqdm import tqdm
import torch
import gc  # gc 패키지 임포트

def calc_rouge(preds, labels):
    preds = preds.argmax(axis=-1)
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    rouge2 = rouge_score.compute(
        predictions=decoded_preds,
        references=decoded_labels
    )
    return rouge2["rouge2"]

def train(model, optimizer, dataloader):
    model.train()
    train_loss = 0.0

    for input_ids, attention_mask, labels in tqdm(dataloader, desc="Training"):
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)

        loss = outputs.loss
        train_loss += loss.item()
        
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # GPU 캐시 비우기
        gc.collect()
        torch.cuda.empty_cache()

    train_loss = train_loss / len(dataloader)
    return train_loss

def evaluation(model, dataloader):
    with torch.no_grad():
        model.eval()
        val_loss, val_rouge = 0.0, 0.0

        for input_ids, attention_mask, labels in tqdm(dataloader, desc="Evaluation"):
            outputs = model(
                input_ids=input_ids, attention_mask=attention_mask, labels=labels
            )
            logits = outputs.logits
            loss = outputs.loss

            logits = logits.detach().cpu().numpy()
            label_ids = labels.to("cpu").numpy()
            rouge = calc_rouge(logits, label_ids)
            
            val_loss += loss
            val_rouge += rouge

        val_loss = val_loss / len(dataloader)
        val_rouge = val_rouge / len(dataloader)
        return val_loss, val_rouge

rouge_score = evaluate.load("rouge", tokenizer=tokenizer)
best_loss = 10000
for epoch in range(epochs):
    train_loss = train(model, optimizer, train_dataloader)
    val_loss, val_accuracy = evaluation(model, valid_dataloader)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.4f} Val Loss: {val_loss:.4f} Val Rouge {val_accuracy:.4f}")

    if train_loss < best_loss:
        best_loss = train_loss
        torch.save(model.state_dict(), f"../Model/Bart_{i}.pt")
        print("Saved the model weights")


Training:   0%|          | 0/353 [00:00<?, ?it/s]C:\Users\mathn\AppData\Roaming\Python\Python38\site-packages\transformers\models\bart\modeling_bart.py:587: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
Evaluation: 100%|██████████| 2/2 [00:00<00:00,  3.49it/s]


Epoch 1: Train Loss: 4.4419 Val Loss: 9.3461 Val Rouge 0.0000
Saved the model weights


Evaluation: 100%|██████████| 2/2 [00:00<00:00,  3.65it/s]


Epoch 2: Train Loss: 4.4233 Val Loss: 9.0943 Val Rouge 0.0000
Saved the model weights


Evaluation: 100%|██████████| 2/2 [00:00<00:00,  3.58it/s]


...

Epoch 24: Train Loss: 4.3786 Val Loss: 11.4615 Val Rouge 0.0000
Saved the model weights


Evaluation: 100%|██████████| 2/2 [00:00<00:00,  3.57it/s]


Epoch 25: Train Loss: 4.3810 Val Loss: 10.7347 Val Rouge 0.0000


Training:  32%|███▏      | 113/353 [00:51<01:48,  2.21it/s]


KeyboardInterrupt: 

In [ ]:
model = BartForConditionalGeneration.from_pretrained(
    pretrained_model_name_or_path="facebook/bart-base"
).to(device)
model.load_state_dict(torch.load(f"../Model/Bart_{i}.pt"))

test_loss, test_rouge_score = evaluation(model, test_dataloader)
print(f"Test Loss : {test_loss:.4f}")
print(f"Test ROUGE-2 Score : {test_rouge_score:.4f}")

OutOfMemoryError: CUDA out of memory. Tried to allocate 20.00 MiB. GPU 

In [7]:
from transformers import pipeline


summarizer = pipeline(
    task="summarization",
    model=model,
    tokenizer=tokenizer,
    max_length=54,
    device="cpu"
)

for index in range(2000,2005):
    news_text = df.text.iloc[index]
    summarization = df.prediction.iloc[index]
    predicted_summarization = summarizer(news_text)[0]["summary_text"]
    print(f"정답 요약문 : {summarization}")
    print(f"모델 요약문 : {predicted_summarization}\n")

정답 요약문 : ガロ
모델 요약문 : �ンンンーンン�ン

정답 요약문 : お迎え
모델 요약문 : �ン�ンンンーンン�

정답 요약문 : 射撃部
모델 요약문 : �ンアンンンーンン�

정답 요약문 : ワンダースワン
모델 요약문 : �ンンン��ン�ン

정답 요약문 : ルパン三世
모델 요약문 : ���ン�ンンンーン

